# Overview
This project aims to analyze aviation accident data from the National Transportation Safety Board (NTSB) to identify the safest aircraft and provide actionable insights that will guide the company’s acquisition strategy.
The analysis will focus on aviation accidents and incidents from 1962 to 2023, covering civil aviation in the United States. The investigation type taken into consideration is accidents. Key areas of investigation will include accident rates by aircraft type, common causes of accidents, and regional risk factors. The goal is to translate these findings into three concrete business recommendations that will help the head of the new aviation division make informed decisions on which aircraft to purchase.

# Business Problem
Your company is expanding in to new industries to diversify its portfolio. Specifically, they are interested in purchasing and operating airplanes for commercial and private enterprises, but do not know anything about the potential risks of aircraft. You are charged with determining which aircraft are the lowest risk for the company to start this new business endeavor. You must then translate your findings into actionable insights that the head of the new aviation division can use to help decide which aircraft to purchase.


#### Stakeholders
The main stakeholder of this project is the Head of the Aviation Division.

# Data
Two datasets were obtained for this project.
* A .csv file from the National Transportation Safety Board that includes aviation accident data from 1962 to 2023 about civil aviation accidents and selected incidents in the United States and international waters.
* A .csv file with the United States names and their abbreviations
#### Data Sources
The data was obtained from Kaggle (https://www.kaggle.com/datasets/khsamaha/aviation-accident-database-synopses)

# Key Business Questions
* Which aircraft makes are associated with the fewest total injuries in recorded accidents?
* What are the most common causes of aviation accidents?
* Which regions  are associated with higher risks?

# Data Cleaning

# 1.0 Importing our Libraries

In [2]:
# Importing the necessary modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


%matplotlib inline

# 1.1  Loading Data

In [ ]:
# Reading our data from a csv into a dataframe
# First Dataframe
Aviation_data_df=pd.read_csv('AviationData.csv', encoding='ISO-8859-1')

# Second Dataframe
States_df=pd.read_csv('USState_Codes.csv', encoding='ISO-8859-1')


In [4]:
Aviation_data=Aviation_data_df.copy(deep=True)

# 1.2 Previewing our Data

#### Aviation_data_df

In [ ]:
# Previewing the first 5 rows
Aviation_data.head()

In [ ]:
# Previewing the last 5 rows
Aviation_data.tail()

# 1.3 Accessing information about our data

In [ ]:
# Dataset information
Aviation_data.info()

In [ ]:
# Summary Statistics of the data
Aviation_data.describe()

In [ ]:
# Shape of the  data
Aviation_data.shape

In [ ]:
# Data types of the columns
Aviation_data.dtypes

#### States_df

In [ ]:
States_df

In [ ]:
# Shape
States_df.shape

In [ ]:
# Summary statistics
States_df.describe()

# 1.4 Data Cleaning
This step is crucial for ensuring data accuracy and consistency, making the dataset ready for analysis.


#### States_df

In [ ]:
# Checking for duplicates
States_df.duplicated().sum()

In [ ]:
# Checking for null values
States_df.isnull().sum()

### Checking for Duplicates

In [ ]:
# Checking for duplicates in the dataframe
duplicates=Aviation_data.duplicated().sum() # This checks for the duplicates and sums them up
print(f'Number of Duplicated rows: {duplicates}')

In [ ]:
# Dropping the duplicates
Aviation_data.drop_duplicates()

### Checking for missing values

In [ ]:
#Checking and summing the  missing values in the dataframe
Aviation_data.isnull().sum()

A few observation columns were dropped since they were not going to be used in the analysis;
* Latitude and Longitude
* Schedule
* Air carrier
* Airport Name
* Airport code
* Publication Date
* Report Status
* Broad phase of flight

In [ ]:
# Dropping irrelevant observations
Aviation_data.drop(columns=['Latitude','Longitude', 'Schedule', 'Air.carrier', 'Airport.Name','Airport.Code',
                               'Publication.Date','Report.Status','Registration.Number','FAR.Description', 'Broad.phase.of.flight'],inplace=True)
#Checking the remaining columns
Aviation_data_df.columns

Cleaning the 'Event.Date' column.
* Converting the date values into datetime object for easier analysis
* Extracting the year

In [ ]:
# Event Date Column
# Convert the values into date format
Aviation_data['Event.Date'] = pd.to_datetime(Aviation_data['Event.Date'])
Aviation_data['Event.Date']

In [ ]:
# Creating a 'Year' column
Aviation_data['Year']=Aviation_data['Event.Date'].dt.year # .dt.year extracts the year from the date
Aviation_data['Year']

In [22]:
# Dropping the Event.Date column
Aviation_data.drop(columns=['Event.Date'], axis=1, inplace=True)

**Injury Severity column**
* Removing the extra characters i.e. (int). The numbers inside the parentheses are captured under 'Total.Fatal.Injuries' column

In [ ]:
#Replacing the () and any value inside the parentheses with an empty string
Aviation_data['Injury.Severity']=Aviation_data['Injury.Severity'].str.replace(r'\(.*\)', '', regex=True).str.strip()
Aviation_data['Injury.Severity'].unique()

**Location Column**
* Converting all values to uppercase letters for uniformity
* Splitting City and State

In [24]:
# Converting all the values to uppercase
Aviation_data['Location']= Aviation_data['Location'].str.upper()

# Split the City and State
# Splitting the value at comma, and accessing the string at index 0
Aviation_data['City'] = Aviation_data['Location'].str.split(',').str[0]

# Splitting the value at comma, and accessing the string at index 1
Aviation_data['State'] = Aviation_data['Location'].str.split(',').str[1]

# Removing characters at the start and end of the values
Aviation_data['State'] = Aviation_data['State'].str.strip()


**Make Column**
* Converting values to uppercase
* Stripping the values of any characters at the start and end
* Harmonizing the names to have less unique values

In [ ]:
# Converting the values into upper case
Aviation_data['Make']=Aviation_data['Make'].str.upper()

# Removing any extra characters at the start and end of each value
Aviation_data['Make'] = Aviation_data['Make'].str.strip(',".')

# Creating function to extract the first word
def extract_first_word(value):
    if isinstance(value, str):  # Check if the value is a string
        return value.split()[0]  # Split and return the first word
    else:
        return value
    # Apply function to 'Make' column
Aviation_data['Make']=Aviation_data['Make'].apply(extract_first_word)
Aviation_data['Make'].unique() # Getting the unique values

**Aircraft Category**
* Harmonizing the data by renaming categories that appear to be the same


In [ ]:
# Aircraft Category

Aviation_data['Aircraft.Category'].replace({'UNK':'Unknown', 'ULTR':'Ultralight', 'WSFT':'Weight-Shift'}, inplace=True)
Aviation_data['Aircraft.Category'].unique()

**Weather Condition**
* Harmonizing values

In [ ]:
Aviation_data['Weather.Condition'] = Aviation_data['Weather.Condition'].replace({'Unk':'Unknown','UNK':'Unknown','Personal':'Unknown'})
Aviation_data['Weather.Condition'].unique()

**Purpose of Flight**
* Harmonizing the value names


In [ ]:
# 'Purpose of flight' column
Aviation_data['Purpose.of.flight'].replace({'Other Work Use':'Unknown', 'Aerial Application':'Aerial Observation',
                'Air Race/show':'Air Race show', 'Business':'Executive/corporate', 'PUBL':'Public', 'Public Aircraft - Local':'Public Aircraft',
                  'Public Aircraft - State':'Public Aircraft' , 'Public Aircraft - Federal':'Public Aircraft','External Load':'Ferry' , 'Banner Tow':'Glider Tow',
                          'Public':'Public Aircraft'}, inplace=True)
Aviation_data['Purpose.of.flight'].unique()

**Engine Type**

In [ ]:
# Engine Type
Aviation_data['Engine.Type']=Aviation_data_df['Engine.Type'].replace({'UNK':'Unknown', 'Geared Turbofan':'Turbo Fan'})
Aviation_data['Engine.Type'].unique()

#### Cleaning the States_df

In [30]:
# Removing characters at the start and end of the values
States_df['Abbreviation'] = States_df['Abbreviation'].str.strip()

## Handling Missing Values

**Injury Severity Column**

In [ ]:
# Fill the missing values with the most common severity
most_injury_severity= Aviation_data['Injury.Severity'].mode()[0] # Most common severity

Aviation_data.fillna({'Injury.Severity':most_injury_severity}, inplace=True)

Aviation_data['Injury.Severity'].value_counts() # Getting the count of the unique values

**Location Column**

In [32]:
# Filling the missing values
Aviation_data['Location'].fillna('Unknown',inplace=True)


**Make Column**

In [33]:
# Filling the missing values with unknown
Aviation_data['Make'].fillna('Unknown', inplace=True)


**Aircraft Category**

In [ ]:
# Finding the most common category
most_common_cat=Aviation_data['Aircraft.Category'].mode()[0]

# Filling the missing values with the most common value
Aviation_data['Aircraft.Category'].fillna(most_common_cat, inplace=True)
Aviation_data['Aircraft.Category'].unique()


**Amateur Built**

In [35]:
# Getting the most common response
most_common_response= Aviation_data['Amateur.Built'].mode()[0]

# Filling the values with the most common response
Aviation_data.fillna({'Amateur.Built':most_common_response}, inplace=True)

**Weather Condition**

In [36]:
# Most common weather condition
most_common_weather=Aviation_data['Weather.Condition'].mode()[0]

# Filling the missing values with the most common
Aviation_data['Weather.Condition'].fillna(most_common_weather, inplace=True)


**Purpose of Flight**

In [37]:
# Filling the missing values with the most common purpose
common_purpose=Aviation_data['Purpose.of.flight'].mode()[0]

# Fill the missing values with the most common purpose
Aviation_data.fillna(common_purpose, inplace=True)


**Total Uninjured Column**

In [38]:
# Filling the NaN values with 0
Aviation_data['Total.Uninjured'].fillna(0, inplace=True)

**Total Fatal Injuries**

In [39]:
# Fill missing values based on Injury Severity
Aviation_data.loc[(Aviation_data['Injury.Severity'] == 'Fatal') &
(Aviation_data['Total.Fatal.Injuries'].isnull()),
    'Total.Fatal.Injuries'
] = 1
Aviation_data.loc[(Aviation_data['Injury.Severity'] == 'Non-Fatal') &
(Aviation_data['Total.Fatal.Injuries'].isnull()),
    'Total.Fatal.Injuries'
] = 0
Aviation_data.loc[(Aviation_data['Injury.Severity'] == 'Incident') &
(Aviation_data['Total.Fatal.Injuries'].isnull()),
    'Total.Fatal.Injuries'
] = 0
Aviation_data.loc[(Aviation_data['Injury.Severity'] == 'Minor') &
(Aviation_data['Total.Fatal.Injuries'].isnull()),
    'Total.Fatal.Injuries'
] = 1
Aviation_data.loc[(Aviation_data['Injury.Severity'] == 'Serious') &
(Aviation_data['Total.Fatal.Injuries'].isnull()),
    'Total.Fatal.Injuries'
] = 1
Aviation_data.loc[(Aviation_data['Injury.Severity'] == 'Unavailable') &
(Aviation_data['Total.Fatal.Injuries'].isnull()),
    'Total.Fatal.Injuries'
] = 0


**Total Serious Injuries Column**

In [40]:
# Filling the missing values with 0
Aviation_data['Total.Serious.Injuries'].fillna(0, inplace=True)

**Total Minor Injuries**

In [41]:
# Filling the missing values with 0
Aviation_data['Total.Minor.Injuries'].fillna(0,inplace=True)

#### Creating a Total Injuries Column
This contains an aggregation of Total Fatal,Serious and Minor injuries

In [ ]:
# Convert the columns to float64
Aviation_data['Total.Minor.Injuries'] = pd.to_numeric(Aviation_data['Total.Minor.Injuries'], errors='coerce')
Aviation_data['Total.Fatal.Injuries'] = pd.to_numeric(Aviation_data['Total.Fatal.Injuries'], errors='coerce')
Aviation_data['Total.Serious.Injuries'] = pd.to_numeric(Aviation_data['Total.Serious.Injuries'], errors='coerce')

# Creating a new column with Total recorded injuries
Aviation_data['Total.Injuries']=(Aviation_data['Total.Fatal.Injuries']+Aviation_data['Total.Serious.Injuries']+
                                    Aviation_data['Total.Minor.Injuries'])

Aviation_data['Total.Injuries']

Dropping the Total fatal,minor and serious injuries. The 'Total.Injuries' column will be used for analysis

In [43]:
# Dropping the Total fatal,minor and serious injuries
Aviation_data.drop(columns=['Total.Minor.Injuries','Total.Fatal.Injuries','Total.Serious.Injuries'], inplace=True)

In [ ]:
# State column
Aviation_data['State'].dropna() # Drop null rows

The research will be limited to United States. The following code filters data within the United States

In [ ]:
# Country column
Aviation_data = Aviation_data[(Aviation_data['Country'] == 'United States')& (Aviation_data['Investigation.Type']=='Accident')]
Aviation_data


## Visualizations

### 1. Which aircraft makes are associated with the fewest total injuries in recorded accidents?
 * Which makes of aircrafts are most common?
 * Among the common makes, which category is most common?
 * Among the common makes, which ones have the lowest accident rates?


**Most Common Makes**

In [ ]:
# Most common aircrafts
# Extracting the sum of the unique values and sorting them in ascending order
top_10_makes=Aviation_data['Make'].value_counts().sort_values(ascending=False)[:10] # Selecting the 10 most common makes
top_10_makes_list=top_10_makes.index.to_list() # Converting the dataframe into a list
top_10_makes_list



In [ ]:
filtered_Aviation_data_df = Aviation_data[Aviation_data['Make'].isin(top_10_makes_list)]
filtered_Aviation_data_df

In [ ]:
# Plotting the data
fig, ax = plt.subplots(figsize=(10, 6))

# Random Colors to use in plotting
colors = np.random.rand(len(top_10_makes_list), 3)

# Create the bar plot
ax.bar(top_10_makes_list,top_10_makes, color=colors)

# Add labels and title
ax.set_xlabel('Aircraft Makes', fontsize=12)
ax.set_ylabel('Number of Occurrences', fontsize=12)
ax.set_title('Top 10 Most Common Aircraft Makes', fontsize=15)

# Rotate the x-axis labels for readability
plt.xticks(rotation=45, ha='right')

# Adjust layout to prevent label cutoff
fig.tight_layout()

# Show the plot
plt.show()

**Observation**
* The top 10 most common aircraft makes involved in accidents are: Cessna, Piper, Beech, Bell, Grumman, Robinson, Mooney, Bellanca, Boeing and Air


**Total Injuries by Make**

In [ ]:
# Grouping dataframe by Make and summing the Total injuries in the grouped data
injuries_by_make = filtered_Aviation_data_df.groupby('Make')['Total.Injuries'].sum()

# Filter the df to inlcude only the aircraft makes present in top_10_makes_list
filtered_injuries = injuries_by_make[top_10_makes_list]

# Print the filtered dataframe
filtered_injuries

In [ ]:
# Plotting Aircraft make and their injuries
# Plot the figure
fig, ax=plt.subplots(figsize=(10,6))

# Bar plot
ax.bar(top_10_makes_list,filtered_injuries, color=colors)

# Set the x-axis label and title
ax.set_xlabel('Top 10 Aircraft Makes')
ax.set_ylabel('Number of Injuries')
ax.set_title('Aircraft Make Versus Total Injuries')

# Rotating the x-axis labels for readability
plt.xticks(rotation=45, ha='right')

# Adjust the layout
fig.tight_layout()

#Show the plot
plt.show()

**Observation**
* CESSNA has the highest number of accidents but also a high number of aircraft in operation, which could skew the perception of risk.

* GRUMMAN, ROBINSON and AIR have lower total injuries, indicating potentially safer records.

**Most Common Category**

In [ ]:
Aviation_data_df['Aircraft.Category'].unique()

In [ ]:
# Top 10 categories 
top_10_categories=Aviation_data_df['Aircraft.Category'].value_counts().sort_values(ascending=False)[:10] # Selecting the 10 categories
top_10_categories_list=top_10_categories.index.to_list() # Converting the dataframe into a list
top_10_categories_list

In [ ]:
# Plot the figure
fig, ax=plt.subplots(figsize=(10,6))

# Plot the bar plot
ax.scatter(top_10_categories_list,top_10_categories, color=colors)

# Set labels and Title
ax.set_xlabel('Aircraft Categories', fontsize=12)
ax.set_ylabel('Number of Occurrences', fontsize=12)
ax.set_title('Top 10 Most Common Aircraft Categories', fontsize=15)

# Rotate the x-axis labels for readability
plt.xticks(rotation=45, ha='right')

# Adjust layout to prevent label cutoff
fig.tight_layout()

# Show the plot
plt.show()

**Observation**
>Airplanes are the most common aircraft category involved in accidents.


In [ ]:
# Aircraft Category by make

top_10_categories_make=filtered_Aviation_data_df.groupby('Make')['Aircraft.Category'].value_counts().sort_values(ascending=False)[:10] # Selecting the 10 categories
top_10_categories_make

In [ ]:
top_10_categories_make_df=pd.DataFrame(top_10_categories_make)
# Extracting 'Make' and 'Aircraft Category' from the MultiIndex for labeling
makes_categories = [f'{make} - {category}' for make, category in top_10_categories_make.index]

# Plotting
plt.figure(figsize=(10,6))

# Horizontal Bar Plot
plt.barh(makes_categories, top_10_categories_make.values, color=colors)

# Set labels and Title
plt.xlabel('Count')
plt.ylabel('Make - Category')
plt.title('Top 10 Aircraft Categories by Make')

**Observation**
* Within the top makes, ROBINSON is notable for being primarily associated with Helicopters.

**Injuries and Aircraft Category**

In [ ]:
category_injuries=filtered_Aviation_data_df.groupby(['Make', 'Aircraft.Category'])['Total.Injuries'].sum().sort_values(ascending=False)[:10]
category_injuries

In [ ]:
# Plotting
plt.figure(figsize=(10,6))

# Horizontal Bar Plot
plt.bar(makes_categories, category_injuries, color=colors)

# Set labels and Title
plt.xlabel('Make-Category')
plt.ylabel('Total Injuries')
plt.title('Injuries by Aircraft Category and Make')

# Rotate the x-axis labels for readability
plt.xticks(rotation=45, ha='right')

**Aircraft Damage**

In [58]:
# Aircraft Damage by make
aircraft_damage= filtered_Aviation_data_df.groupby(['Make','Aircraft.damage']).size().unstack(fill_value=0)
aircraft_damage

# Filter the df to inlcude only the aircraft makes present in top_10_makes_list
filtered_damage = aircraft_damage.loc[top_10_makes_list]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

# Plot each damage category as a stacked bar
ax.bar(top_10_makes_list, filtered_damage['Destroyed'], label='Destroyed', color='#FF9100')
ax.bar(top_10_makes_list, filtered_damage['Substantial'], bottom=filtered_damage['Destroyed'], label='Substantial', color=colors[1])
ax.bar(top_10_makes_list, filtered_damage['Minor'],
       bottom=filtered_damage['Destroyed'] + filtered_damage['Substantial'], label='Minor', color=colors[2])

# Set the x-axis label, y-axis label, and title
ax.set_xlabel('Top 10 Aircraft Makes')
ax.set_ylabel('Number of Damage Cases')
ax.set_title('Aircraft Make Versus Total Damage')

# Rotate the x-axis labels for readability
plt.xticks(rotation=45, ha='right')

# Add a legend to distinguish the damage types
ax.legend()

# Adjust layout to prevent overlap
fig.tight_layout()

# Show the plot
plt.show()

**Observation**
> All aircrafts makes underwent substantial damage after the accident

## Causes of Aviation Accidents.
We will consider:
* Weather condition
* Amateur Built
* Purpose of the flight


In [ ]:
# Filtering the top 10 makes from the original dataframe
filtered_df = filtered_Aviation_data_df[filtered_Aviation_data_df['Make'].isin(top_10_makes_list)]

# injuries by make and weather condition
weather_injury= filtered_df.groupby(['Make', 'Weather.Condition'])['Total.Injuries'].sum().unstack()

# Plot the figure and axes
fig, ax = plt.subplots(figsize=(12, 8))

# Plotting each make's injuries by weather condition
weather_injury.plot(kind='bar', ax=ax)

# Add labels and title
ax.set_xlabel('Make')
ax.set_ylabel('Total Injuries')
ax.set_title('Total Injuries by Make and Weather Condition')

# Rotating the x-axis labels for readability
plt.xticks(rotation=45, ha='right')

# Adjust layout
fig.tight_layout()

# Show the plot
plt.show()

**Observation**
>A majority of accidents occur under Visual Meteorological Conditions (VMC) rather than Instrument Meteorological Conditions (IMC) or Unknown conditions.
>Injuries sustained during VMC are significantly higher across all top aircraft makes.
>CESSNA and PIPER have higher injuries under VMC, suggesting other factors like human error or mechanical issues.

In [ ]:
# Amateur Built
# Injuries by make andAmateur Built
Amateur_injuries=filtered_Aviation_data_df.groupby(['Make','Amateur.Built'])['Total.Injuries'].sum().unstack()
Amateur_injuries



In [ ]:
# Plot the stacked bar chart
Amateur_injuries.plot(kind='bar', stacked=True, figsize=(12, 8))

# Set title and labels
plt.title('Total Injuries by Make and Amateur Built (Stacked)')
plt.xlabel('Aircraft Make')
plt.ylabel('Total Injuries')


# Rotate x-axis labels for better readability
plt.xticks(rotation=45, ha='right')

# Show the plot
plt.tight_layout()
plt.show()

**Observation**
> The majority of accidents involved professionally manufactured aircrafts, given they are more common, but Amateur Built aircraft may have higher risk due to variability in construction quality

In [ ]:
# Purpose of the flight

# Injuries by make and purpose of flight
purpose_injuries=filtered_Aviation_data_df.groupby(['Make','Purpose.of.flight'])['Total.Injuries'].sum().unstack()
purpose_injuries


In [ ]:
# Plot the stacked bar chart
purpose_injuries.plot(kind='bar', stacked=True, figsize=(12, 8))

# Rotate x-axis labels for better readability
plt.xticks(rotation=45, ha='right')

# Set title and labels
plt.title('Total Injuries by Make and Purpose of Flight (Stacked)')
plt.xlabel('Make')
plt.ylabel('Total Injuries')

# Show the plot
plt.tight_layout()
plt.show()

**Observation**
>The most common purpose at the time of accidents is Personal flights.
>Instructional flights also show a significant number of accidents, highlighting the risks during training.

### Regions associated with higher risks
* Which regions have high number of injuries?

In [ ]:
filtered_states= Aviation_data[Aviation_data['State'].isin(States_df['Abbreviation'])]
filtered_states.shape

In [ ]:
# Group data by region
# Group by 'State' and sum the 'Total Injuries'
injuries_by_state =filtered_states.groupby('State')['Total.Injuries'].sum()

# Sort the results and select the top 10
sorted_injuries_by_state = injuries_by_state.sort_values(ascending=False).head(10)
sorted_injuries_by_state


In [ ]:
# Plot the total injuries by state
fig, ax = plt.subplots(figsize=(12, 8))

# Plot
sorted_injuries_by_state.head(10).plot(kind='bar', ax=ax, color=colors)

# Set labels and title
ax.set_xlabel('State')
ax.set_ylabel('Total Injuries')
ax.set_title('Top 10 States with Highest Total Injuries')

# Rotate x-axis labels for better readability
plt.xticks(rotation=45, ha='right')

# Show the plot
plt.tight_layout()
plt.show()

**Observation**
* The States of California(CA), Texas(TX)  and Florida(FL) have the highest number of fatalities while Washington(WA) has the lowest injuries/fatalities

In [68]:
cleaned_data=filtered_Aviation_data_df.to_excel('Cleaned_Aviation_Data.xlsx', index=False)